## Tennis: AI Playing

Here, we load the trained agents, and let the agents play some games.

In [1]:
from unityagents import UnityEnvironment
import numpy as np

In [1]:
# Load the environent
env = UnityEnvironment(file_name="Tennis.app")

# Get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# Reset the environment, this time train_mode is false
env_info = env.reset(train_mode=False)[brain_name]


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


In [8]:
# Number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# Size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# Examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('Size of each state:', state_size)

Number of agents: 2
Size of each action: 2
Size of each state: 24


We load the agent class and the the saved weights into the multiagent instance.

We now load the weights.


In [4]:
import torch
from multi_afrom multigent_ddpg import MultiAgent
multiagent = MultiAgent(num_agents, state_size, action_size, seed=2345)

# Load the weights
for i, agent in enumerate(multiagent.agents):
    agent.actor_local.load_state_dict(torch.load(f'checkpoint_actor_{i}.pth'))
    agent.critic_local.load_state_dict(torch.load(f'checkpoint_critic{i}.pth'))

Let us play 10 games, and see the individual and aggregate performance. Recall that our trained multiagent obtained an average score over 100 epsiodes of 1.29.

In [5]:
n_ep = 10
scores_ep = []
for i in range(1, n_ep + 1):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = multiagent.act(states, 0) # Noise is zero
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))
    scores_ep.append(np.max(scores))
print(f'Average maximum over all the epsiodes: {np.mean(scores_ep)}')

Score (max over agents) from episode 1: 2.1000000312924385
Score (max over agents) from episode 2: 0.10000000149011612
Score (max over agents) from episode 3: 2.600000038743019
Score (max over agents) from episode 4: 0.7000000104308128
Score (max over agents) from episode 5: 1.8000000268220901
Score (max over agents) from episode 6: 0.4000000059604645
Score (max over agents) from episode 7: 1.0000000149011612
Score (max over agents) from episode 8: 0.0
Score (max over agents) from episode 9: 2.600000038743019
Score (max over agents) from episode 10: 1.1000000163912773
Average maximum over all the epsiodes: 1.2400000184774398


We can see that in 10 games, the distribution of the scores is very unstable, but the average is still within the expected range. Finally, we close the environment, so that we avoid leaving it hanging. 

In [6]:
env.close()